In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from utils import misc


df = pd.read_csv('datasets/daily_price_series/SPY.csv')
df = misc.clean_df(df)
df = df.set_index('date')
df

,open,high,low,close,adjclose,volume,ticker
date,,,,,,,
1993-01-29,43.968750,43.968750,43.750000,43.937500,25.122364,1003200,SPY
1993-02-01,43.968750,44.250000,43.968750,44.250000,25.301012,480500,SPY
1993-02-02,44.218750,44.375000,44.125000,44.343750,25.354612,201300,SPY
1993-02-03,44.406250,44.843750,44.375000,44.812500,25.622643,529400,SPY
1993-02-04,44.968750,45.093750,44.468750,45.000000,25.729851,531500,SPY
...,...,...,...,...,...,...,...
2023-05-02,414.769989,414.820007,407.820007,410.839996,410.839996,103998500,SPY
2023-05-03,411.359985,413.869995,407.769989,408.019989,408.019989,91531800,SPY
2023-05-04,406.929993,407.269989,403.739990,405.130005,405.130005,94901900,SPY


In [2]:
random_orders = np.array([0 if f <= 0.5 else 1 for f in np.random.rand(len(df))])
np.mean(random_orders)

0.4899645808736718

In [6]:

class backtester:
    def __init__(self, **kwargs) -> None:
        self.df = None
        self.initial_capital = misc.get_attr(kwargs, 'initial_capital', 1E6)
        self.reward_function = misc.get_attr(kwargs, 'reward_function', None)

        
        self.clean_slate()

        


    def clean_slate(self):
        '''
        Set initial default state of environment.
        Called by __init__ and reset
        '''
        # State variables
        self.cash = self.initial_capital
        self.position = 0
        self.position_value = 0
        self.portfolio_value = self.cash + self.position_value
        self.leverage = abs(self.position_value / self.portfolio_value)
        self.portfolio_volatility = 0
        self.portfolio_return = 1
        self.end = False
        self.current_step = 0
        self.cost_basis = 0

        # record all instantaneous values of state
        self.record = {
            'cash': [],
            'position': [],
            'position_value': [],
            'portfolio_value': [],
            'leverage': [],
            'portfolio_volatility': [],
            'portfolio_return': [],
        }

        # record all trades
        self.trade_record = {
            'order_price': [],
            'order_quantity': [],
            'execution_price': [],
            'execution_quantity': [],
            # 'cost_basis': [],
        }

        # instantaneous state of the trader
        self.trader_state = np.array([
            self.cash,
            self.position,
            self.position_value,
            self.portfolio_value,
            # self.leverage,
            # self.portfolio_volatility,
        ])



    def set_asset(self, ticker=None, df_path=None, update=False):
        if ticker is None:
            self.df = pd.read_csv(df_path)
        else:
            try:
                self.df = pd.read_csv('datasets/daily_price_series/'+ ticker +'.csv')
            except FileNotFoundError:
                pass
            if self.df is None or update:
                self.df = misc.get_price_data(ticker)
        
        self.df = misc.clean_df(self.df)
        self.df = self.df.set_index('date')

        self.open = self.df['open'].to_numpy()
        self.high = self.df['high'].to_numpy()
        self.low = self.df['low'].to_numpy()
        self.close = self.df['close'].to_numpy()
        self.adjclose = self.df['adjclose'].to_numpy()



    def __take_action(self, order_quantity=0, order_price=0, order_type='market_on_close'):
        execution_price = 0
        execution_quantity = 0
        evaluation_price = self.adjclose[self.current_step]
        if order_quantity == 0:
            pass
        else:
            if order_type == 'market_on_close':
                execution_price = self.adjclose[self.current_step]
                execution_quantity = order_quantity
            
            
            self.position += order_quantity
            self.cash -= execution_price * execution_quantity

            if self.position != 0:
                self.cost_basis = execution_price
            else:
                self.cost_basis = 0
            
            self.position_value = self.position * self.cost_basis


        




        
            

        

    
bt = backtester()
bt.set_asset('GME')
bt.df

,open,high,low,close,adjclose,volume,ticker
date,,,,,,,
2002-02-13,2.406250,2.515000,2.381250,2.51250,1.691666,76216000,GME
2002-02-14,2.543750,2.548750,2.481250,2.50000,1.683250,11021600,GME
2002-02-15,2.500000,2.506250,2.462500,2.48750,1.674834,8389600,GME
2002-02-19,2.475000,2.475000,2.343750,2.38750,1.607504,7410400,GME
2002-02-20,2.400000,2.468750,2.381250,2.46875,1.662210,6892800,GME
...,...,...,...,...,...,...,...
2023-05-03,18.559999,19.200001,18.530001,18.90000,18.900000,3184000,GME
2023-05-04,19.250000,19.480000,18.920000,19.35000,19.350000,2041800,GME
2023-05-05,19.620001,20.879999,19.510000,20.42000,20.420000,4353700,GME
